In [ ]:
import JupyterFormatter
JupyterFormatter.enable_autoformat();

In [ ]:
import Glob
import JLD2
import ProgressMeter: @showprogress

In [ ]:
PART = 0
NUM_PARTS = 1
SPLIT = ""

In [ ]:
include("import_lists.jl");

In [ ]:
function get_userid_map()
    df = read_csv(get_data_path("processed_data/userid_map.csv"))
    df[!, :userid] = parse.(Int32, df.userid)
    ret = Dict{String,Dict{String,Int32}}(s => Dict() for s in Set(df.source))
    @showprogress for (s, username, userid) in zip(df.source, df.username, df.userid)
        ret[s][username] = userid
    end
    ret
end;

In [ ]:
function get_max_valid_ts(f)
    files = Glob.glob("raw_$(f)/*/user_media_facts/user_status.*.csv", get_data_path(""))
    max_valid_ts =
        maximum([maximum(parse.(Int64, read_csv(f).access_timestamp)) for f in files])
    @assert max_valid_ts <= Dates.datetime2unix(Dates.now())
    max_valid_ts
end;

In [ ]:
function import_lists(sourcedir, outdir, userid_map, max_valid_ts)
    if !ispath(outdir)
        mkpath(outdir)
    end
    for m in ["manga", "anime"]
        for s in ["mal", "anilist", "kitsu", "animeplanet"]
            files = Glob.glob("$s/user_media_facts/user_$(m)_list.*.csv", sourcedir)
            files =
                [x for (i, x) in Iterators.enumerate(sort(files)) if i % NUM_PARTS == PART]
            @showprogress for f in files
                stem = basename(f)[1:end-length(".csv")]
                fn = "$outdir/$s.$stem.jld2"
                df = Dict(
                    "data" =>
                        import_list(m, s, userid_map[s], max_valid_ts, read_csv(f)),
                )
                JLD2.save(fn, df; compress = false)
            end
        end
    end
end;

In [ ]:
const USERID_MAP = get_userid_map();
f = "$(SPLIT)_data"
if ispath(get_data_path("raw_$(f)"))
    import_lists(
        get_data_path("raw_$(f)"),
        get_data_path(f),
        USERID_MAP,
        get_max_valid_ts(f),
    )
end